In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

binary_data_path = '/content/drive/MyDrive/NLP_Workspace/Data/'

df = pd.read_csv(binary_data_path + "emoji_labeled_data_nonltk_BERT.csv")
df = df.dropna(how='any',axis=0)

In [ ]:
df.sample(10)

,label,text
93668,7,isfp
64654,5,problem positive typing opposed negative typin...
22987,1,told many different people intimidating seems ...
161871,13,music psychology makeup & fashion video games ...
3458,0,results table listing percentage number
165159,13,hello everyone looooong time away site returne...
159308,12,richard russ hudson riso
10400,0,way nothing take personally discovered values ...
25542,2,ritalin bullshit high estimate physiological g...
115202,9,avoided writing anything threads feel really s...


In [ ]:
df = pd.concat([df['label'], df['text']], axis=1)

In [ ]:
DOWNSAMPLE = 50000
df_balanced = df.sample(DOWNSAMPLE)
df_balanced['label'].value_counts()

9     3184
8     3180
6     3169
2     3154
12    3152
1     3145
3     3133
4     3130
0     3122
15    3111
10    3110
5     3101
11    3099
14    3091
13    3085
7     3034
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_balanced, test_size=0.15)

In [ ]:
train.head(3)

,label,text
168609,13,honestly think actually sick give call couple ...
20158,1,l fav guess mellow near cold-hearted l possibl...
26602,2,understand post


In [ ]:
test.head(3)

,label,text
148771,11,pansexual fall love somebody gender happily re...
103520,8,okay play thread game want say wtf hokahey def...
25876,2,quite entjs admit wrong long logical agrument ...


In [ ]:
ourpur_dir = binary_data_path + "BERT_data/16class_BERT_withEmojis/"
train.to_csv(ourpur_dir+"16class_train_data.csv", index=False)
test.to_csv(ourpur_dir+"16class_test_data.csv", index=False)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 182 kB 72.7 MB/s 
     |████████████████████████████████| 212 kB 66.9 MB/s 
     |████████████████████████████████| 132 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 80.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
data_files = {"train": ourpur_dir+"16class_train_data.csv", "test": ourpur_dir+"16class_test_data.csv"}

from datasets import load_dataset
squad_it_dataset = load_dataset("csv", data_files=data_files, sep=',')

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-96cd5c9accf22ad9/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 42500
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 7500
    })
})

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 78.6 MB/s 


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [ ]:
TRAIN_NUM = int(DOWNSAMPLE * 0.85)
TEST_NUM = int(DOWNSAMPLE * 0.15)
TEST_NUM

7500

In [ ]:
train_dataset = squad_it_dataset["train"].shuffle(seed=42).select([i for i in list(range(TRAIN_NUM))])
test_dataset = squad_it_dataset["test"].shuffle(seed=42).select([i for i in list(range(TEST_NUM))])

tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)

  0%|          | 0/42500 [00:00<?, ?ex/s]

  0%|          | 0/7500 [00:00<?, ?ex/s]

In [ ]:
tokenized_train[0]

{'label': 0,
 'text': 'mean yum circles lol',
 'input_ids': [101, 2812, 9805, 2213, 7925, 8840, 2140, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=16)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [ ]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='micro')["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
training_args = TrainingArguments(
    
    output_dir= ourpur_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 42500
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 13285
  Number of trainable parameters = 66965776
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster

Step,Training Loss
500,2.761300
1000,2.685800
1500,2.637900
2000,2.612800
2500,2.604500
3000,2.543400
3500,2.497800
4000,2.487900
4500,2.484200
5000,2.441500


Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT_withEmojis/checkpoint-500
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT_withEmojis/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT_withEmojis/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT_withEmojis/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT_withEmojis/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT_withEmojis/checkpoint-1000
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT_withEmojis/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/16class_BERT_wi

TrainOutput(global_step=13285, training_loss=2.238565324202578, metrics={'train_runtime': 1022.1362, 'train_samples_per_second': 207.898, 'train_steps_per_second': 12.997, 'total_flos': 1651079062760064.0, 'train_loss': 2.238565324202578, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7500
  Batch size = 16


<ipython-input-23-dfe153c31386>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 2.4723312854766846,
 'eval_accuracy': 0.26786666666666664,
 'eval_f1': 0.26786666666666664,
 'eval_runtime': 9.2036,
 'eval_samples_per_second': 814.898,
 'eval_steps_per_second': 50.958,
 'epoch': 5.0}